# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
!jupyter nbextension enable --py gmaps


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [15]:
%matplotlib inline

In [16]:
!pip install gmaps

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
city_data_to_load = "citydata.xlsx"
city_data = pd.read_csv(city_data_to_load)


city_data[['Latitude', 'Longitude']].dtypes



Latitude     float64
Longitude    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
gmaps.configure(api_key=g_key)

In [21]:
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
idealtemp_df=city_data.loc[city_data['Max Temp F'] >60]
idealtemp_df=city_data.loc[city_data['Max Temp F'] <75]
idealwind_df=idealtemp_df.loc[idealtemp_df["Wind Speed mph"] <10]
idealweather_df=idealwind_df.loc[idealwind_df["Humidity"]<40]


idealweather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Country,Cloudiness,Date,Max Temp F,Wind Speed mph
47,47,Nikolskoye,59.70,30.79,14.00,30,4.00,RU,33,1590348252,57.20,8.95
55,55,Kumul,42.80,93.45,20.60,30,2.83,CN,100,1590348324,69.08,6.33
96,96,Central Point,42.38,-122.92,21.67,37,3.10,US,1,1590348260,71.01,6.93
117,117,Khorixas,-20.37,14.97,19.81,29,1.48,NaN,0,1590348066,67.66,3.31
134,134,Saint George,37.10,-113.58,23.00,12,4.10,US,1,1590348112,73.40,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df=idealweather_df
hotel_df["Hotel Name"] = ""


hotel_df["Latitude"] = hotel_df["Latitude"].map("{:.0f}".format)
hotel_df['Latitude'] = hotel_df['Latitude'].astype(str).astype(int)

hotel_df["Longitude"] = hotel_df["Latitude"].map("{:.0f}".format)
hotel_df['Latitude'] = hotel_df['Latitude'].astype(str).astype(int)
#df["a"] = df["a"].astype(str).astype(int)

#hotel_df['Latitude'].astype(np.int64)
#hotel_df = hotel_df['Longitude'].astype(int)
#df['column name'] = df['column name'].astype(np.int64)
hotel_df.dtypes

C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

Unnamed: 0          int64
City               object
Latitude            int32
Longitude          object
Max Temp          float64
Humidity            int64
Wind Speed        float64
Country            object
Cloudiness          int64
Date                int64
Max Temp F        float64
Wind Speed mph    float64
Hotel Name         object
dtype: object

In [11]:
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

KeyError: 'Lat'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
